In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Convolution2D, Flatten, Dropout,GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
from keras.applications.inception_resnet_v2 import preprocess_input
from attention import AttentionLayer
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
#import keras
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

2022-02-15 01:34:43.756395: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

2022-02-15 01:34:44.907483: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-02-15 01:34:44.933133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-15 01:34:44.933704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:24:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.86GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2022-02-15 01:34:44.933731: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-02-15 01:34:44.935530: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-02-15 01:34:44.937270: I tensorflow/stream_executor/plat

In [3]:
datadir = "./data/akac_gen.txt"
dataset = pd.read_csv(datadir, delimiter ="\t", header = 0)
#print(list(dataset.columns))
dataset = dataset.sample(frac=1, random_state=77)  # row 전체 shuffle

SNPs = ['rs2235373','rs16873348','rs2284791','rs2013162','rs595918','rs3753582','rs2235375','rs1044516','rs11204737','rs1044516','rs7715100','rs3917192','rs2235371']
full = list(dataset.columns)
del full[0:6]
full.remove('rs11466414')
full.remove("rs12089548")
print(full)
snps = ['rs11204737', 'rs2255796', 'rs2235373','rs16873348']
#,'rs2284791','rs595918','rs3753582','rs2013162', 'rs1139'
#['rs2235373','rs16873348','rs2284791','rs2013162','rs595918','rs3753582']
#dataset.head()

['rs3737967', 'rs1537514', 'rs2274976', 'rs1801131', 'rs1801133', 'rs9651118', 'rs3753582', 'rs4147871', 'rs560426', 'rs1209169', 'rs481931', 'rs16827741', 'rs2228099', 'rs11204737', 'rs3768012', 'rs7517566', 'rs1044516', 'rs599021', 'rs17317411', 'rs2235373', 'rs2235371', 'rs2235375', 'rs595918', 'rs17015218', 'rs2013162', 'rs3821272', 'rs930655', 'rs3732247', 'rs765871', 'rs3771498', 'rs3771497', 'rs3755377', 'rs3771485', 'rs11466212', 'rs3771475', 'rs1139', 'rs3821949', 'rs12532', 'rs4464513', 'rs698', 'rs1693482', 'rs2241894', 'rs2255796', 'rs7713638', 'rs7715100', 'rs15251', 'rs16873348', 'rs910586', 'rs2819861', 'rs1934328', 'rs987525', 'rs7078160', 'rs7940667', 'rs7103685', 'rs931953', 'rs10790330', 'rs906830', 'rs7129848', 'rs3935406', 'rs10892434', 'rs4409845', 'rs2136421', 'rs2295221', 'rs7142363', 'rs2073247', 'rs17104928', 'rs17176643', 'rs11156925', 'rs17104939', 'rs17104944', 'rs17104965', 'rs1884213', 'rs3917211', 'rs2284791', 'rs3917205', 'rs4252340', 'rs3917192', 'rs39

In [4]:
dataset.shape
y_train = pd.to_numeric(dataset['affected']) -1
X_train = dataset[snps]
x_train_np = np.array(X_train).astype("float")
y_train_np = np.array(y_train)
input_dim = X_train.shape[1]
print(x_train_np.shape)

(262, 4)


In [5]:
snps_num = 16
no_of_generations = 200
no_of_individuals = 30
mutate_factor = 0.03

In [6]:
def mutate(new_individual):
    genes = []
    leftover = [v for v in full if v not in new_individual]
    for gene in new_individual:
        n = random.random()
        if(n < mutate_factor):
            k = random.randrange(len(leftover))
            genes.append(leftover[k])
            del leftover[k]
        else:
            genes.append(gene)  
    return genes

def crossover(individuals):
    new_individuals = []
    elite = 8
    for i in range(elite):
        new_individuals.append(individuals[i])
    #엘리트 살아남음

    for i in range(elite, no_of_individuals):
        new_individual = []
        #if(i < (no_of_individuals - 2)):
        if(i == 2):
            parentA = random.choice(individuals[:3])
            parentB = random.choice(individuals[:3])
        else:
            parentA = random.choice(individuals[:])
            parentB = random.choice(individuals[:])
        #아래에서 유전자 반반씩 나눠 줌
        for i in range(len(parentA)):
            n = random.random()
            if(n< 0.5):
                if parentA[i] in new_individual:
                    new_individual.append(parentB[i])
                else:
                    new_individual.append(parentA[i])
            else:
                if parentB[i] in new_individual:
                    new_individual.append(parentA[i])
                else:
                    new_individual.append(parentB[i])
        #else:
        #    new_individual = random.choice(individuals[:])

        new_individuals.append(mutate(new_individual))
        #new_individuals.append(new_individual)

    return new_individuals

def evolve(individuals, fitness):
  
    sorted_y_idx_list = sorted(range(len(fitness)),key=lambda x:fitness[x])
    individuals = [individuals[i] for i in sorted_y_idx_list ]
    fitness = [fitness[i] for i in sorted_y_idx_list ]
    #print(fitness)
    #individuals.reverse()

    new_individuals = crossover(individuals)

    return new_individuals

In [7]:
def load_model(size, input_dim):
    model = Sequential()
    model.add(Dense(size,input_dim = input_dim,activation='relu'))
    model.add(Dense(size*2,input_dim = input_dim,activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(size*2,input_dim = size,activation='relu'))
    model.add(Dense(1,input_dim = size,activation='sigmoid', name='lastfc'))
    model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])
    
    return model

In [8]:
def train(individuals):
    fitness = []
    fitness_a = []
    for i in range(len(individuals)):
        model.set_weights(Wsave)
        X_train = dataset[individuals[i]]
        x_train_np = np.array(X_train).astype("float")
        
        history = model.fit(x=X_train,y=y_train, epochs=100,verbose=0,validation_split=0.25)
        
        for k,j in zip(history.history["val_loss"], history.history["loss"]):
            fitness_a.append((k+j)/2)  
        fitness.append(round(min(fitness_a), 4))
        
    return individuals, fitness

In [10]:
individuals = []
for i in range(no_of_individuals):
    individuals.append(random.sample(full, snps_num))
    
# control loop
size = 32
model = load_model(size, snps_num)
Wsave = model.get_weights()
best = []
best_fitness = 10
best_generation = 0

for generation in range(no_of_generations):
    
    individuals, fitness = train(individuals)
    print(fitness)
    
    if best_fitness >= min(fitness):
        best.append(individuals[fitness.index(min(fitness))])
        best_fitness = min(fitness)
        best_generation = generation+1
        
    if generation == no_of_generations -1:
        break       
    individuals = evolve(individuals, fitness)

2022-02-15 01:34:45.029366: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-15 01:34:45.053960: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3599530000 Hz
2022-02-15 01:34:45.054921: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x25a0450 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-15 01:34:45.054955: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-02-15 01:34:45.129729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-0

[0.2174, 0.1653, 0.1653, 0.1653, 0.1653, 0.1653, 0.1653, 0.1653, 0.1653, 0.1653, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648, 0.1648]
[0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646, 0.1646]
[0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616, 0.1616]
[0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673, 0.1673]
[0.1849, 0.1849, 0.1849, 0.1849, 0.1

KeyboardInterrupt: 

In [11]:
print(best)
print(best_fitness)
print(best_generation)
print(individuals)

[['rs3771485', 'rs3821272', 'rs195291', 'rs4464513', 'rs481931', 'rs3917211', 'rs17104939', 'rs11156925', 'rs599021', 'rs2013162', 'rs2284791', 'rs931953', 'rs4252328', 'rs7129848', 'rs15251', 'rs2274976'], ['rs3771485', 'rs3821272', 'rs195291', 'rs4464513', 'rs481931', 'rs3917211', 'rs17104939', 'rs11156925', 'rs599021', 'rs2013162', 'rs2284791', 'rs931953', 'rs4252328', 'rs7129848', 'rs15251', 'rs2274976'], ['rs3771485', 'rs3821272', 'rs195291', 'rs4464513', 'rs481931', 'rs3917211', 'rs17104939', 'rs11156925', 'rs599021', 'rs2013162', 'rs2284791', 'rs931953', 'rs4252328', 'rs7129848', 'rs15251', 'rs2274976'], ['rs3771485', 'rs1537514', 'rs2235371', 'rs4252328', 'rs7129848', 'rs1044516', 'rs4464513', 'rs11796677', 'rs2819861', 'rs1934328', 'rs3821949', 'rs16827741', 'rs17317411', 'rs1801133', 'rs481931', 'rs11796677'], ['rs3771485', 'rs2235371', 'rs2013162', 'rs4252328', 'rs7129848', 'rs7940667', 'rs4464513', 'rs16873348', 'rs2819861', 'rs1934328', 'rs2284791', 'rs16827741', 'rs173174